# get normalized output and denormalize it according to the models in models folder

In [26]:
import math
import locale
import numpy as np
import pandas as pd
import plotly.express as px
from tqdm import tqdm
from joblib import dump, load
import pickle
STATION = "Parco S. Chiara"
pollu = ["Biossido di Azoto", "Ossido di Carbonio", "PM10", "Ozono", "PM2.5", "Biossido Zolfo"]
POLLUTANT = "O3"

In [27]:
pollu_dict = {
    "AO2": "Biossido di Azoto",
    "CO2": "Ossido di Carbonio",
    "PM10": "PM10",
    "O3": "Ozono",
    "PM2.5": "PM2.5",
    "SO2": "Biossido Zolfo"
}
MODEL_FILE = f"models/{pollu_dict[POLLUTANT]}.joblib"
FILE = "data/test_day_data.csv"

In [28]:
try:
    model = load(MODEL_FILE)
except KeyError:
    model = pickle.load(open(MODEL_FILE.split('.')[0]+'.sav', 'rb'))
day_data = pd.read_csv(FILE)
# read an entire day of normalized output data, you need the entire day

In [29]:
day_data.Data = pd.to_datetime(day_data.Data, format = "%Y-%m-%d %H:%M:%S")
sum = day_data.Valore.sum()

In [30]:
day_of_year = day_data.at[day_data.index[1],'Data'].day_of_year

In [31]:
prediction_sum = model.predict([[day_of_year]])
print(prediction_sum)

[259.26164449]


In [32]:

day_data["real_ppm"]= day_data.Valore * prediction_sum

test = pd.read_csv("data/APPA_data_merged.csv")
test.Data = pd.to_datetime(test.Data, format = "%Y-%m-%d %H:%M:%S")
test = test[test.Stazione == STATION]
test = test[test.Inquinante == pollu_dict[POLLUTANT]]
test[test.Data.dt.date == test.at[test.index[200],'Data'].date()].to_csv("data/test_day_data.csv")

In [33]:
day_data.to_csv(FILE)